In [1]:
import $ivy.`org.apache.lucene:lucene-core:7.2.1`
import $ivy.`org.apache.lucene:lucene-queries:7.2.1`
import $ivy.`org.apache.lucene:lucene-queryparser:7.2.1`
import $ivy.`org.apache.lucene:lucene-analyzers-common:7.2.1`

import $ivy.$                                    

import $ivy.$                                       

import $ivy.$                                           

import $ivy.$                                                

In [2]:
import org.apache.lucene.analysis.standard.StandardAnalyzer
import org.apache.lucene.document._
import org.apache.lucene.index._
import org.apache.lucene.store.RAMDirectory
import org.apache.lucene.util.BytesRef

import org.apache.lucene.analysis.standard.StandardAnalyzer

import org.apache.lucene.document._

import org.apache.lucene.index._

import org.apache.lucene.store.RAMDirectory

import org.apache.lucene.util.BytesRef

We'll use this to convert some nasty iterator-like Java objects to Scala collections.

In [3]:
def toScalaStream[T](iter: {def next(): T}): Stream[T] = {
  val value = iter.next()
  if (value == null) Stream.empty[T]
  else value #:: toScalaStream(iter)
}

defined function toScalaStream

## The usual setup

In [4]:
val analyzer = new StandardAnalyzer()
val indexWriter = new IndexWriter(
  new RAMDirectory(),
  new IndexWriterConfig(analyzer))

analyzer: StandardAnalyzer = org.apache.lucene.analysis.standard.StandardAnalyzer@18eb897b
indexWriter: IndexWriter = org.apache.lucene.index.IndexWriter@55b71374

## TODO

In [5]:
val textFieldType = new FieldType()

textFieldType.setIndexOptions(IndexOptions.DOCS_AND_FREQS_AND_POSITIONS)
textFieldType.setTokenized(true)
textFieldType.setStored(true)
textFieldType.setStoreTermVectors(true)

textFieldType: FieldType = stored,indexed,tokenized,termVector

In [6]:
textFieldType.docValuesType

res5: DocValuesType = NONE

In [7]:
val contents = Array(
  "Humpty Dumpty sat on a wall,",
  "Humpty Dumpty had a great fall.",
  "All the king's horses and all the king's men",
  "Couldn't put Humpty together again."
)

contents: Array[String] = Array(
  "Humpty Dumpty sat on a wall,",
  "Humpty Dumpty had a great fall.",
  "All the king's horses and all the king's men",
  "Couldn't put Humpty together again."
)

In [8]:
contents.foreach { content =>
  val doc = new Document();
  val textField = new Field("content", content, textFieldType)
  
  doc.add(textField)
  indexWriter.addDocument(doc)
}

indexWriter.commit()

res7_1: Long = 7L

In [9]:
val indexReader = DirectoryReader.open(
  indexWriter.getDirectory())

indexReader: DirectoryReader = StandardDirectoryReader(segments_1:4 _0(7.2.1):c4)

In [12]:
(0 until indexReader.maxDoc) foreach { i =>
  
  val terms = indexReader.getTermVector(i, "content")
  val termsIterator = terms.iterator()
  val termsStream = toScalaStream(terms.iterator()).map(_.utf8ToString())
  println(s"Document $i")
  termsStream.foreach { term =>
    print(term + " ")
  }
  println()
}

Document 0
dumpty humpty sat wall 
Document 1
dumpty fall great had humpty 
Document 2
all horses king's men 
Document 3
again couldn't humpty put together 
